Using https://ensemble-pytorch.readthedocs.io/en/latest/parameters.html#voting

In [1]:
import sys

sys.path.insert(1, "../")

from nets.SimpleNet.lib.simple_model import CNN
from nets.SimpleNet.config.torch_config import get_transform
from nets.SimpleNet.utils.video_dataset import VideoFrameDataset
from nets.SimpleNet.config.dataset import get_dataset_path
from nets.SimpleNet.utils.loader import split_dataset


In [2]:
NUM_SEGMENTS = 10
FRAMES_PER_SEGMENT = 5
BATCH_SIZE = 16
IMAGE_SIZE = 224
IMAGE_RANDOM_CROP_RESIZE = 0.8
NUM_EPOCHS = 20
LEARNING_RATE = 1e-3
DEBUG = False


In [3]:
data_path, model_path = get_dataset_path(dataset="WLASL/videos", model_name="WLASL")

In [4]:
multiple_transform = get_transform(IMAGE_SIZE, IMAGE_RANDOM_CROP_RESIZE)

In [5]:
dataset = VideoFrameDataset(
    root_path=data_path,
    transform=multiple_transform,
    num_segments=NUM_SEGMENTS,
    frames_per_segment=FRAMES_PER_SEGMENT,
)

classes = dataset.classes
print(classes)


['before', 'book', 'candy', 'chair', 'clothes', 'computer', 'cousin', 'drink', 'go', 'who']


In [6]:
train_loader, test_loader, validation_loader = split_dataset(
    dataset, train_split=0.70, validation_split=0.1, batch_size=BATCH_SIZE
)


In [7]:
from torchensemble import VotingClassifier

ensemble = VotingClassifier(
    estimator=CNN,
    estimator_args={
        "num_classes": len(classes),
        "batch_size": BATCH_SIZE,
        "num_frames": FRAMES_PER_SEGMENT * NUM_SEGMENTS,
        "image_size": int(IMAGE_SIZE * IMAGE_RANDOM_CROP_RESIZE),
        "debug": DEBUG,
    },
    n_estimators=BATCH_SIZE,
    cuda=True,
)


In [8]:
from torch import nn, optim
from torchensemble.utils.set_module import set_scheduler

In [9]:
ensemble.set_criterion(nn.CrossEntropyLoss())

In [10]:
# optimizer = optim.Adam(ensemble.parameters(), lr=LEARNING_RATE)
ensemble.set_optimizer("Adam", lr=LEARNING_RATE)

In [11]:
# ensemble.set_scheduler(
#     "ReduceLROnPlateau",
#     mode="min",
#     min_lr=1e-6,
#     factor=0.7,
#     patience=5,
# )

ensemble.set_scheduler(
    "CosineAnnealingLR",
    T_max=NUM_EPOCHS,
    eta_min=1e-6,
    last_epoch=-1,
)


In [12]:
ensemble.fit(
    train_loader=train_loader,
    test_loader=validation_loader,
    epochs=NUM_EPOCHS,
)


Estimator: 000 | Epoch: 000 | Batch: 000 | Loss: 2.32312 | Correct: 1/16
Estimator: 000 | Epoch: 001 | Batch: 000 | Loss: 2.03627 | Correct: 3/16
Estimator: 000 | Epoch: 002 | Batch: 000 | Loss: 1.87788 | Correct: 5/16
Estimator: 000 | Epoch: 003 | Batch: 000 | Loss: 1.75680 | Correct: 6/16
Estimator: 000 | Epoch: 004 | Batch: 000 | Loss: 2.02489 | Correct: 4/16
Estimator: 000 | Epoch: 005 | Batch: 000 | Loss: 1.66316 | Correct: 6/16
Estimator: 000 | Epoch: 006 | Batch: 000 | Loss: 1.85634 | Correct: 4/16
Estimator: 000 | Epoch: 007 | Batch: 000 | Loss: 1.86088 | Correct: 7/16
Estimator: 000 | Epoch: 008 | Batch: 000 | Loss: 2.15056 | Correct: 3/16
Estimator: 000 | Epoch: 009 | Batch: 000 | Loss: 1.89540 | Correct: 4/16
Estimator: 000 | Epoch: 010 | Batch: 000 | Loss: 1.68008 | Correct: 7/16
Estimator: 000 | Epoch: 011 | Batch: 000 | Loss: 1.84478 | Correct: 7/16
Estimator: 000 | Epoch: 012 | Batch: 000 | Loss: 2.26831 | Correct: 1/16


In [ ]:
accuracy = ensemble.evaluate(test_loader)
print(f"Accuracy: {accuracy}")


Accuracy: 10.526315789473685
